In [1]:
import sys
from pathlib import Path

#Path.cwd().parent depends on where you run this code from
#Path(__file__).resolve().parents[1] depends on where this file is located on disk
#
#project_root = .../1099-reconciliation-pipeline
project_root = Path.cwd().parent  # running from notebooks/  folder (cwd = current working directory) - .parent gets us to the folder above
sys.path.append(str(project_root)) #sys.path is a list of folders where python looks for modules
                                    # we add the project root to that list with .append

print(project_root)

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline


In [ ]:
from src.clean_relius import clean_relius
from src.clean_matrix import clean_matrix
from src.match_transactions import reconcile_relius_matrix
from src import load_data
from src.config import RAW_DATA_DIR

relius_path = RAW_DATA_DIR / "real_relius_2025.xlsx"
matrix_path = RAW_DATA_DIR / "real_matrix_2025.xlsx"

# Load raw data to DataFrames
relius_raw = load_data.load_relius_excel(path=relius_path, use_sample_if_none=False)
matrix_raw = load_data.load_matrix_excel(path=matrix_path, use_sample_if_none=False)

# Clean DataFrames
relius_clean = clean_relius(relius_raw)
matrix_clean = clean_matrix(matrix_raw)

# Only inherited plans for now:
inherited_plans = ["300004PLAT", "300004MBD", "300004MBDII"]

matched = reconcile_relius_matrix(
    relius_clean,
    matrix_clean,
    plan_ids=inherited_plans,
    apply_business_rules=True,
)

.shape is an attribute of pandas DataFrames that returns a tuple of
# (number of rows, number of columns) - e.g. (1000, 15)
matched.shape

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline/src/clean_matrix.py:96: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(series, errors="coerce").dt.date


(664, 36)

In [4]:
matched.head(50)

,plan_id,ssn,first_name,last_name,state_relius,gross_amt,exported_date,tax_year,dist_code_1,dist_name,...,match_status,expected_tax_code_1,expected_tax_code-1,expected_tax_code_2,code_matches_expected,needs_correction,suggested_tax_code_1,suggested_tax_code_2,correction_reason,action
0,300004MBD,150406376,NaN,NaN,NaN,151452.18,NaT,<NA>,NaN,NaN,...,unmatched_matrix,<NA>,<NA>,NaN,False,False,<NA>,<NA>,<NA>,<NA>
1,300004MBD,150406376,Robert,Wilkins,DE,151522.18,2025-10-15,0,G,Rollover,...,unmatched_relius,<NA>,<NA>,NaN,True,False,<NA>,<NA>,<NA>,<NA>
2,300004MBD,159364560,Maria,Cantalupo,PA,10872.69,2025-10-02,0,4,RMD ACH,...,perfect_match,4,<NA>,<NA>,True,False,<NA>,<NA>,<NA>,<NA>
3,300004MBD,186545616,Maureen,King,PA,25000.00,2025-09-22,0,4,Partial Liquidation Net ACH,...,perfect_match,4,<NA>,<NA>,True,False,<NA>,<NA>,<NA>,<NA>
4,300004MBD,188368532,NaN,NaN,NaN,1699.02,NaT,<NA>,NaN,NaN,...,unmatched_matrix,<NA>,<NA>,NaN,False,False,<NA>,<NA>,<NA>,<NA>
5,300004MBD,188368532,Deborah,Gryshuk,PA,1709.02,2025-10-03,0,7,Recurring Check Net,...,unmatched_relius,<NA>,<NA>,NaN,True,False,<NA>,<NA>,<NA>,<NA>
6,300004MBD,194420270,NaN,NaN,NaN,669.52,NaT,<NA>,NaN,NaN,...,unmatched_matrix,<NA>,<NA>,NaN,False,False,<NA>,<NA>,<NA>,<NA>
7,300004MBD,194420270,David,Symons,PA,2652.38,2025-06-13,0,4,Partial Liquidation Gross ACH,...,match_needs_correction,4,<NA>,<NA>,False,True,4,<NA>,inherited_cash_expected_4,UPDATE_1099
8,300004MBD,194420270,NaN,NaN,NaN,2678.10,NaT,<NA>,NaN,NaN,...,unmatched_matrix,<NA>,<NA>,NaN,False,False,<NA>,<NA>,<NA>,<NA>
9,300004MBD,194420270,David,Symons,PA,3347.62,2025-06-11,0,4,RMD ACH,...,unmatched_relius,<NA>,<NA>,NaN,True,False,<NA>,<NA>,<NA>,<NA>


In [ ]:
# How many times each Matrix transaction id appears in the merged df
tx_id_counts = matched["transaction_id"].value_counts()

# Show only those that appear more than once
tx_id_counts[tx_id_counts > 1].head(20)